In [1]:
### Importing all libraries

import smtplib as smt
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
#import boto3
from datetime import datetime
from datetime import timedelta
from io import StringIO
import os
#import pymysql


yesterday_date = datetime.today() - timedelta(1)
yesterday_date_str1 = yesterday_date.strftime("%Y-%m-%d")
yesterday_date_str2 = yesterday_date.strftime("%Y/%m/%d")

### Making a database connection & fetching data; Change the credentails in case required 

#pe_dl_dbname = os.environ.get("SKULL_DBNAME")
#pe_dl_host = os.environ.get("SKULL_HOST")
#pe_dl_port = os.environ.get("SKULL_PORT")
#pe_dl_user = os.environ.get("SKULL_USER")
#pe_dl_password = os.environ.get("SKULL_PASSWORD")


#db1 = create_engine('postgresql://'+ pe_dl_user +':'+ pe_dl_password+'@'+pe_dl_host+':'+'5439'+'/' +pe_dl_dbname)

db1 = psycopg2.connect(host='dp-prod-redshift.cvsmnufhrlzh.ap-south-1.redshift.amazonaws.com',user='shubhamsingh',port=5439,password='2A3Epae93y',dbname='skull')

curr = db1.cursor()


In [2]:
### ucode list

query_txt = open('query.txt','r')
df = pd.read_sql_query(query_txt.read(), con=db1)


In [4]:
email_df = df

df.head()

,action_date,classification,actioned_by,action_task,action_point,total_actions,total_orders_delivered
0,2019-10-01,CnR,karandeep_kaur,reassign,5,353,299.0
1,2019-10-01,STC,abdul_shaikh,STC_Hold,2,119,NaN
2,2019-10-01,STC,sumaiya_shaikh,STC_Hold,2,116,NaN
3,2019-10-01,CnR,divya_acharya,reassign,5,103,88.0
4,2019-10-01,SOH,Hemangi_Vaity,Hold_other,1,98,NaN


In [5]:

### Sending feedback email to stake-holders

def format_html(df, title=''):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
        text-align: left;
        font-family: Helvetica, Arial, sans-serif;
    }
    table {
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        align: middle;
        text-align: left;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%;
    }

</style>
</head>
<body>
    '''
    result += '<h3> %s </h3>\n' % title
    result += df.to_html(index = False, na_rep=0,float_format='{0:.0f}'.format, classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    return result



##### EMAILING THE TABLES #####
import email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

excelname = 'SubscriptionAgentEfficiency' + yesterday_date_str1 + '.xlsx'
email_df.to_excel(excelname,index=False,sheet_name='Data')

me = "reporting@pharmeasy.in"
#cc = "analytics@pharmeasy.in"
you = "rahul.aranha@pharmeasy.in" #"sanyoug@pharmeasy.in,suraj@pharmeasy.in,floorwalker_cc@pharmeasy.in,soniya.lal@pharmeasy.in,jeffie.joseph@pharmeasy.in,swati.khera@pharmeasy.in,sagar.salvi@pharmeasy.in,dheeraj@pharmeasy.in,dhiraj@pharmeasy.in,analytics@pharmeasy.in"

attachments = 'SubscriptionAgentEfficiency' + yesterday_date_str1 + '.xlsx'

msg = MIMEMultipart('alternative')
msg['Subject'] = 'Subscription Agent Efficiency - ' + yesterday_date_str2
msg['From'] = me
#msg["Cc"] = cc
msg['To'] = you

with open(attachments, 'rb') as a_file:
	basename = os.path.basename(attachments)
	part = MIMEApplication(a_file.read(), Name=basename) 

part['Content-Disposition'] = 'attachment; filename="%s"' % basename

htmlbody = '<body><br>Team,<br><br>This email provides a raw dump of <b>Agent Tasks</b> for <b>current month till yesterday</b></body><br>'

htmlbody = htmlbody + '\n' + '<br><br>Regards,<br>Analytics Team'

emailbody = (htmlbody)
part2 = MIMEText(emailbody, 'html')

msg.attach(part)
msg.attach(part2)

s = smt.SMTP_SSL('smtp.gmail.com', 465)
s.login(me, "26B5922F2BC94SZ1ME4B158474BF9")
s.sendmail(me,you.split(','),msg.as_string())
s.quit()

print("Automated Email Sent")

Automated Email Sent
